Определение языка (language detection)
--------------------

Александра Степанова. БКЛ-151

### Первый метод: частотные слова

#### Скачаем немного википедии для тестов
Воспользуемся пакетом *wikipedia*:

`pip install wikipedia`

In [7]:
import wikipedia
import codecs
import collections
import sys
import re

Возьмем французский, исландский, шведский и литовский языки.

In [2]:
langs = ('fr', 'is', 'sv', 'lt')

In [3]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [4]:
wiki_texts = {}
for lang in langs: 
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

c:\users\user\appdata\local\programs\python\python35-32\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file c:\users\user\appdata\local\programs\python\python35-32\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Trastos (homonymie)
Skipping page Jamie Smith (football)
Skipping page Danny Watts
Skipping page Sadievo
Skipping page Au Petit Paris
fr 95
Skipping page Örn
Skipping page Hnakki
Skipping page Línuskip
is 97
Skipping page Shortys Island
Skipping page Sochiapa
Skipping page Darah-ye Shākh-e Safēd
Skipping page Collins Hill
Skipping page Déré (vattendrag i Centralafrikanska republiken, lat 5,55, long 16,39)
Skipping page Sungai Bunut
Skipping page Vincenzo Montefusco
Skipping page Laguna El Chino
Skipping page Kasuni
Skipping page Isoluoto (del av en ö)
Skipping page Roß Kopf
sv 89
Skipping page Gaižiūnų apylinkė
Skipping page Rimšelis
Skipping page Levinas
Skipping page Ptakai (reikšmės)
Skipping page Girdauskas
Skipping page Santa Krusas
Skipping page Solnikai (reikšmės)
lt 93


In [6]:
print(wiki_texts['is'][0]) # распечатаем пару текстов, чтобы убедиться, что все хорошо
print(wiki_texts['sv'][0])

Laddi
Þórhallur Sigurðsson (fæddur 20. janúar 1947), best þekktur sem Laddi, er íslenskur leikari, söngvari, tónskáld og skemmtikraftur. Hann hefur gefið út plötur, leikið í kvikmyndum og tekið þátt í miklum fjölda skemmtiþátta sem handritshöfundur og leikari, t.d Heilsubælinu, Imbakassanum og Spaugstofunni Einnig hefur hann leikið í mörgum Áramótaskaupum og tekið þátt í að semja þau. Hann hefur skapað fjöldann allan af karakterum sem margir kannast við og nægir að nefna Eirík Fjalar, Saxa lækni, Skúla rafvirkja, Magnús bónda, Ho Si Mattana, Elsu Lund, Martein Mosdal og svo mætti lengi telja. Kvikmyndir sem hann hefur leikið í eru t.d Stella í orlofi, Stella í framboði, Magnús, Regína, Íslenski draumurinn, Jóhannes, Ófeigur gengur aftur og fleiri. Einnig hefur hann starfað mikið í leikhúsi, en frægustu hlutverk hans á þeim vettvangi eru líklega Fagin í Óliver Twist og Tannlækninn í Litlu Hryllingsbúðinni.
Laddi var í tvíeykinu Halli og Laddi ásamt bróður sínum Haraldi Sigurðssyni. Þeir

Чистим тексты от всех небуквенных единиц.

In [53]:
def tokenize(text):
    pattern = re.compile('[\.,\-\`\#\?\+\*\^\d\=\~\!\@\$\%\&\(\)\_\—\[\]\{\}\:\;\'\"\/\\\>\<\|„“]')
    text = pattern.sub(' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.split(' ')

Создаем словарь, в котором будут храниться все частотные списки. Сначала создаем частотные сеты для каждого языка, избавляемся от пересечений, после чего помещаем их в словарь, который и будет использовать для определения языка. 

In [54]:
def make_freq(lang, wiki_texts):
    freq_lang = collections.defaultdict(lambda: 0)
    corpus = wiki_texts[lang]
    for article in corpus:
        for word in tokenize(article.replace('\n', '').lower()):
            freq_lang[word] += 1
    return set(freq_lang.keys())

In [55]:
is_set = make_freq('is', wiki_texts)
fr_set = make_freq('fr', wiki_texts)
sv_set = make_freq('sv', wiki_texts)
lt_set = make_freq('lt', wiki_texts)

In [57]:
duplicates = set.intersection(is_set, fr_set, sv_set, lt_set)

In [58]:
is_set = is_set - duplicates
fr_set = fr_set - duplicates
sv_set = sv_set - duplicates
lt_set = lt_set - duplicates

In [59]:
lt_set

{'fildingas',
 'mokslų',
 'elektrinio',
 'stilių',
 'kitos',
 'kidulių',
 'petrosianą',
 'stogas',
 'turkiją',
 'van',
 'užsibuvusi',
 'kelerius',
 'masės',
 'bėgdamas',
 'buen',
 'aplaistytas',
 'manila',
 'nerija',
 'galvojamas',
 'grupėmis',
 'lobio',
 'šilalės',
 'tepirmavo',
 'pastatų',
 'buožių',
 'pilotas',
 'einantis',
 'verakruso',
 'hobitai',
 'teritorijų',
 'dardanelių',
 'miliamperų',
 'vietose',
 'kovą',
 'operos',
 'pilnas',
 'pasiruošė',
 'kelis',
 'skaidrus',
 'vaidmenis',
 'raštų',
 'elijadė',
 'dobšina',
 'privalomas',
 'gajumą',
 'švedijoje',
 'botviniku',
 'ტბა',
 'fakultetų',
 'ko',
 'praktikuotas',
 'turnyras',
 'kretingos',
 'viename',
 'kitaip',
 'nešė',
 'strategines',
 'darbai',
 'chelsea',
 'smarkiai',
 'kitų',
 'geografas',
 'paieška',
 'automobilis',
 'keturiolika',
 'pati',
 'laimę…',
 'veikloje',
 'dauguma',
 'valdžia',
 'įsiviešpatavo',
 'fabriką',
 'sugihara',
 'nusipelniusio',
 'elektrėnų',
 'vokietija',
 'suvienijo',
 'songo',
 'mokslo',
 'raj',
 'pla

In [51]:
freqs = {'is': is_set, 'fr': fr_set, 'sv': sv_set, 'lt': lt_set}

Определяем язык.

In [61]:
def predict_language_freqword(text, dic_freqword, langs):
    text_tokens = set(tokenize(text.lower()))
    similar_stat = {}
    
    for lang in langs:
        similar_stat[lang] = len(text_tokens & dic_freqword[lang])

    return max(similar_stat, key=lambda x: similar_stat[x])

### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [74]:
from itertools import islice, tee

def make_ngrams(text):
    pattern = re.compile('[\.,\-\`\#\?\+\*\^\d\=\~\!\@\$\%\&\(\)\_\—\[\]\{\}\:\;\'\"\/\\\>\<\|„“]')
    text = pattern.sub(' ', text)
    text = re.sub(r'\s+', ' ', text)
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    ngrams = [x.strip() for x in ngrams if len(re.findall('\s', x)) == 0] #вычистила "разорванные" пробелами н-граммы
                                                                          #не знаю, насколько это прагматично  
    return ngrams

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [76]:
def make_freq_ngrams(lang, wiki_texts):
    ngrams_lang = collections.defaultdict(lambda: 0)
    corpus = wiki_texts[lang]
    for article in corpus:
        for word in make_ngrams(article.replace('\n', '').lower()):
            ngrams_lang[word] += 1
    return set(ngrams_lang.keys())

In [79]:
is_set_ngs = make_freq_ngrams('is', wiki_texts)
fr_set_ngs = make_freq_ngrams('fr', wiki_texts)
sv_set_ngs = make_freq_ngrams('sv', wiki_texts)
lt_set_ngs = make_freq_ngrams('lt', wiki_texts)

In [80]:
duplicates_ngrams = set.intersection(is_set_ngs, fr_set_ngs, sv_set_ngs, lt_set_ngs)

In [81]:
is_set_ngs = is_set_ngs - duplicates_ngrams
fr_set_ngs = fr_set_ngs - duplicates_ngrams
sv_set_ngs = sv_set_ngs - duplicates_ngrams
lt_set_ngs = lt_set_ngs - duplicates_ngrams

In [82]:
freqs_ngrams = {'is': is_set_ngs, 'fr': fr_set_ngs, 'sv': sv_set_ngs, 'lt': lt_set_ngs}

In [83]:
def predict_language_freqngrams(text, dic_freqngram, langs):
    text_ngrams = set(make_ngrams(text.lower()))
    similar_stat = {}
    
    for lang in langs:
        similar_stat[lang] = len(text_ngrams & dic_freqngram[lang])

    return max(similar_stat, key=lambda x: similar_stat[x])

Тестирование.

In [87]:
text_predictions = {}

for lang in langs: 
    text_predictions[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(text_predictions[lang]))

c:\users\user\appdata\local\programs\python\python35-32\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file c:\users\user\appdata\local\programs\python\python35-32\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Comté d'Owen
Skipping page F̧
Skipping page CEEA
Skipping page Assemblée de la République
Skipping page Volonté
Skipping page Ligne 57
Skipping page Roy Stuart
fr 93
Skipping page Skeið
Skipping page Sigurður Sigurðarson
Skipping page Sergíus 3.
is 97
Skipping page Beluga Mountain
Skipping page Hubbs (auktor)
Skipping page Ämtasjön
Skipping page La Muneca
Skipping page Santa Cruz Buena Vista
Skipping page Sungai Luang
Skipping page Ferreiras (olika betydelser)
Skipping page Sungai Kemubu
Skipping page Waterloo Park
sv 91
Skipping page Ustronė
Skipping page Dubingiai (reikšmės)
Skipping page Arabija
Skipping page Litwa
Skipping page Adomonis
Skipping page Laužikas
lt 94


In [104]:
def test_predictions(text_predictions, langs, dic_freqword, dic_freqngram):
    ngrams = []
    fre = []
    errors_n = []
    errors_f = []
    error_texts = []
    for lang in langs:
        for el in text_predictions[lang]:
            f = predict_language_freqword(el, dic_freqword, langs)
            n = predict_language_freqngrams(el, dic_freqngram, langs)
            ngrams.append(n == lang)
            fre.append(f == lang)
            if n != lang:
                errors_n.append(str(lang) + '-' + str(n))
                error_texts.append(str(lang) + '\n' + el)
            if f != lang:
                errors_f.append(str(lang) + '-' + str(f))
                error_texts.append(str(lang) + '\n' + el)
    return fre, ngrams, errors_n, errors_f, error_texts

In [105]:
fre, ngrams, errors_n, errors_f, error_texts = test_predictions(text_predictions, langs, freqs, freqs_ngrams)

In [95]:
False in fre

False

In [98]:
False in ngrams

True

Посмотрим, какие два языка перепутал метод нграмм и в каких случаях. Для этого немного изменим функцию test_predictions, чтобы записывалась информация об ошибках.

In [103]:
errors

['fr-lt']

Ошибка в одном тексте: французский определился как литовский методом нграмм. Можно попробовать посмотреть на этот текст.

In [108]:
print(error_texts[0])

fr
Liste des villes de Californie du sud par population
Portail de la Californie


Текст маленький и не по большей части состоит из "интернациональных" лексем, что логично объясняет ошибку алгоритма.